In [1]:
from model_funcs import dataset
from model_funcs import train_and_test
from model_funcs import model
import os
from torch.utils.data import DataLoader
import torch
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
from torchvision import models
from torchinfo import summary

from model_funcs.model import ConvolutionalNet

test = dataset.SpectraDataset('test')
loader_test = DataLoader(test, batch_size= 1, pin_memory= True, shuffle=True)


In [3]:
model = models.get_model('shufflenet_v2_x2_0',weights = 'DEFAULT' )

In [2]:
model = ConvolutionalNet()

Downloading: "https://download.pytorch.org/models/mobilenet_v2-7ebf99e0.pth" to /home/alexandre/.cache/torch/hub/checkpoints/mobilenet_v2-7ebf99e0.pth
100%|██████████| 13.6M/13.6M [00:01<00:00, 8.37MB/s]


In [3]:
model

ConvolutionalNet(
  (adaptador): Conv2d(1, 3, kernel_size=(4, 4), stride=(1, 1))
  (base_model): MobileNetV2(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0

In [4]:
for name, params in model.named_parameters():
    print(name, params.requires_grad)

adaptador.weight True
adaptador.bias True
base_model.features.0.0.weight False
base_model.features.0.1.weight False
base_model.features.0.1.bias False
base_model.features.1.conv.0.0.weight False
base_model.features.1.conv.0.1.weight False
base_model.features.1.conv.0.1.bias False
base_model.features.1.conv.1.weight False
base_model.features.1.conv.2.weight False
base_model.features.1.conv.2.bias False
base_model.features.2.conv.0.0.weight False
base_model.features.2.conv.0.1.weight False
base_model.features.2.conv.0.1.bias False
base_model.features.2.conv.1.0.weight False
base_model.features.2.conv.1.1.weight False
base_model.features.2.conv.1.1.bias False
base_model.features.2.conv.2.weight False
base_model.features.2.conv.3.weight False
base_model.features.2.conv.3.bias False
base_model.features.3.conv.0.0.weight False
base_model.features.3.conv.0.1.weight False
base_model.features.3.conv.0.1.bias False
base_model.features.3.conv.1.0.weight False
base_model.features.3.conv.1.1.weight

In [5]:
summary(model)

Layer (type:depth-idx)                                  Param #
ConvolutionalNet                                        --
├─Conv2d: 1-1                                           51
├─MobileNetV2: 1-2                                      --
│    └─Sequential: 2-1                                  --
│    │    └─Conv2dNormActivation: 3-1                   (928)
│    │    └─InvertedResidual: 3-2                       (896)
│    │    └─InvertedResidual: 3-3                       (5,136)
│    │    └─InvertedResidual: 3-4                       (8,832)
│    │    └─InvertedResidual: 3-5                       (10,000)
│    │    └─InvertedResidual: 3-6                       (14,848)
│    │    └─InvertedResidual: 3-7                       (14,848)
│    │    └─InvertedResidual: 3-8                       (21,056)
│    │    └─InvertedResidual: 3-9                       (54,272)
│    │    └─InvertedResidual: 3-10                      (54,272)
│    │    └─InvertedResidual: 3-11                      (5

In [2]:
model = torch.load('models/convolutionalnet.pth')

/tmp/ipykernel_50047/774777495.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('models/convolutionalnet.pth')


In [15]:
X , env = test[int(np.random.uniform(0,200))]
print(X)
model.eval()
with torch.no_grad():
    pred = model(X.unsqueeze(0))

tensor([[[-109.9882, -109.9974, -109.9997,  ..., -109.9988, -109.9910,
          -109.9956],
         [-109.9883, -109.9974, -109.9911,  ..., -109.9904, -109.9906,
          -109.9956],
         [-109.9898, -109.9974, -109.9886,  ..., -109.9878, -109.9913,
          -109.9958],
         ...,
         [-110.0000, -110.0000, -110.0000,  ..., -110.0000, -110.0000,
          -110.0000],
         [-110.0000, -110.0000, -110.0000,  ..., -110.0000, -110.0000,
          -110.0000],
         [-110.0000, -110.0000, -110.0000,  ..., -110.0000, -110.0000,
          -110.0000]]])


In [16]:
env[0] - pred[0][0],  env[1] - pred[0][1]

(tensor(-0.6519), tensor(0.4285))

In [5]:
env[0] , pred[0][0]

(tensor(0.2283), tensor(1.6164))

In [17]:
env[0] *(400-250) + 250 , pred[0][0]*(400-250) + 250

(tensor(399.8198), tensor(497.6096))

In [17]:
env[1] , pred[0][1]

(tensor(0.2360), tensor(0.5185))

In [18]:
env[1] * (0.8 - 0.01) + 0.01 , pred[0][1]* (0.8 - 0.01) + 0.01

(tensor(0.5117), tensor(0.1732))